<a href="https://colab.research.google.com/github/adas754/Basic-Deep-Learning-Practical-implementation/blob/main/age_gender_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

 97% 322M/331M [00:02<00:00, 145MB/s]
100% 331M/331M [00:02<00:00, 129MB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [5]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [6]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [7]:
len(age)

23708

In [8]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [9]:
df.shape

(23708, 3)

In [ ]:
df.head()

,age,gender,img
0,1,1,1_1_3_20161219224454728.jpg.chip.jpg
1,35,0,35_0_3_20170119201122781.jpg.chip.jpg
2,35,0,35_0_0_20170117120747082.jpg.chip.jpg
3,30,0,30_0_0_20170119180216577.jpg.chip.jpg
4,30,1,30_1_2_20170104022925822.jpg.chip.jpg


In [10]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [11]:
train_df.shape

(20000, 3)

In [12]:
test_df.shape

(3708, 3)

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [15]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [16]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 1s 0us/step


In [17]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [18]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [19]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [20]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 259s 398ms/step - loss: 103.4767 - age_loss: 15.3696 - gender_loss: 0.8900 - age_mae: 15.3696 - gender_accuracy: 0.5109 - val_loss: 85.2325 - val_age_loss: 15.1997 - val_gender_loss: 0.7074 - val_age_mae: 15.1997 - val_gender_accuracy: 0.5213
Epoch 2/10
625/625 [==============================] - 249s 398ms/step - loss: 84.6391 - age_loss: 15.1031 - gender_loss: 0.7024 - age_mae: 15.1031 - gender_accuracy: 0.5220 - val_loss: 85.5165 - val_age_loss: 16.9806 - val_gender_loss: 0.6923 - val_age_mae: 16.9806 - val_gender_accuracy: 0.5213
Epoch 3/10
625/625 [==============================] - 248s 396ms/step - loss: 83.5072 - age_loss: 14.8846 - gender_loss: 0.6932 - age_mae: 14.8846 - gender_accuracy: 0.5228 - val_loss: 83.2981 - val_age_loss: 14.7641 - val_gender_loss: 0.6923 - val_age_mae: 14.7641 - val_gender_accuracy: 0.5213
Epoch 4/10
625/625 [==============================] - 249s 399ms/step - loss: 83.5638 - age_loss: 14.8761 - gen